O primeito passo é importar todos os pacotes que você ira utilizar neste tutorial

In [2]:
import numpy as np
from rnn_utils import *

# 1 - Propoagação para frente para Redes Neurais Recorrentes básicas

Nos próximo tutorias você será capaz de gerar música utilizando RNN. A RNN básica que você irá implementar tem a estrutura abaixo. Neste exemplo, $T_x = T_y$. 

<img src="images/RNN.png" style="width:500;height:300px;">
<caption><center> **Figure 1**: Basic RNN model </center></caption>

Veja os passos para implementar uma RNN: 

**Passos**:
1. Implementar os cálculos necessários para computar um passo no tempo da RNN.
2. Implementar o loop sobre os passos $T_x$ para processar na sequencia todas as entradas, uma por vez. 

Vamos lá!

## 1.1 - Célula RNN

Uma Rede Neural Recorrente pode ser entendida com a repetição de uma célula mais simples. A Ideia é você primeiro implementar a computação necessária para um único passo no tempo. A figura abaixo descreve as operações de uma célula RNN em um único passo de tempo.

<img src="images/rnn_step_forward.png" style="width:700px;height:300px;">
<caption><center> **Figura 2**: Uma célula RNN básica. Pega uma entrada (input) $x^{\langle t \rangle}$ (atual input) e $a^{\langle t - 1\rangle}$ (estado anterior escondido contendo informação do passadp), e as saídas (outputs) $a^{\langle t \rangle}$ que é levada para a próxima célula RNN e também utilizada para prever $y^{\langle t \rangle}$ </center></caption>

**Exercício**: Implementar a Célula-RNN descrita na figura (2).

**Instruções**:
1. Computar o estado escondido com a função de ativação tanh (tangente hiperbólica): $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.
2. Utiliza seu novo estado escondido $a^{\langle t \rangle}$, para computar a previsão $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$. Nós forneceremos a você a função: `softmax`.
3. Armazene $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters)$ em cache
4. Retorne $a^{\langle t \rangle}$ , $y^{\langle t \rangle}$ e o cache

Nós iremos vetorizar os $m$ exemplos. Então, $x^{\langle t \rangle}$ terá a dimensão $(n_x,m)$, e $a^{\langle t \rangle}$ terá a dimensão $(n_a,m)$. 

In [3]:
# FUNÇÃO CLASSIFICADA: rnn_cell_forward

def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implementa uma único passo pra frente da célual-RNN como descrito na Figura (2)

    Argumentos:
    xt -- seu dado de entrada no tempo "t", um vetor numpy de dimensões (n_x, m).
    a_prev -- Estado oculto no tempo "t-1", um vetor numpy de dimensões (n_a, m)
    parameters -- dicionário python contendo:
                        Wax -- Matriz de Pesos que multipliaca a entrada (input), vetor numpy de dimensões (n_a, n_x)
                        Waa -- Matriz de Pesos que multipliaca o estado escondido, vetor numpy de dimensões (n_a, n_a)
                        Wya -- Matriz de Pesos que relaciona o estado escondido com a saída (output), vetor numpy de dimensões (n_y, n_a)
                        ba --  Bias, vetor numpy de dimensões (n_a, 1)
                        by -- Bias que relacion o estado escondido com a saída (output), vetor numpy de dimensões (n_y, 1)
    Retorno:
    a_next -- o próximo estado escondido da célula, de dimensões (n_a, m)
    yt_pred -- Previsão no tempo "t", vetor numpy de dimensões (n_y, m)
    cache -- Tupla de valores necessários para o passo para trás, contendo (a_next, a_prev, xt, parameters)
    """
    
    # Recupere os parâmetros de "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### INCIE O SEU CÓDIGO AQUI ### (≈2 lines)
    # Compute a próximo estado de ativação usando a fórmula dada acima
    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    # Compute a saída da célula atual usando a fórmula dada acima
    yt_pred = softmax(np.dot(Wya, a_next) + by)  
    ### SEU CÓDIGO FINALIZA AQUI ###
    
    # Guarder os valores necessários para a propagação para trás na variável cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [4]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
yt_pred.shape =  (2, 10)


**Saída Esperada**: 

<table>
    <tr>
        <td>
            **a_next[4]**:
        </td>
        <td>
           [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
        </td>
    </tr>
        <tr>
        <td>
            **a_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>

</table>

## 1.2 - Passo a frente RNN 

Podemos ver uma RNN como a repetição da célula que você acabou de construir. Se os dados da sequência de entrada são carregados por 10 passos de tempo, então você copiará a sua célula RNN, 10 veses. Cada célula pega como entrada o estado escondido da célula anterior ($a^{\langle t-1 \rangle}$) e o dado de entrada no tempo t atual ($x^{\langle t \rangle}$). Sua saída é o estado oculto ($a^{\langle t \rangle}$) e a previsão ($y^{\langle t \rangle}$) para o passo de tempo atual.


<img src="images/rnn2.png" style="width:800px;height:300px;">
<caption><center> **Figure 3**: RNN Básica. A sequencia de entrada $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  é propagada em $T_x$ passos de tempo. A saída da rede $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>



**Exercício**: Código para a propagação da RNN descrita na Figura (3).

**Instruções**:
1. Cria um vetor de zeros ($a$) que armazenará todos os estados ocultos da RNN.
2. Inicialize o "next" estado escondido como $a_0$ (estado escondido inicial).
3. Itere sobre cada passo de tempo, com seu indice incremental de $t$ :
    - Atualize o "next" estado oculto e o cache executando `rnn_cell_forward`
    - Armazene o "next" estado oculto em  $a$ na posição ($t^{th}$) 
    - Armazene a previsão em y
    - Adicione o cache para a lista de caches
4. Retorne $a$, $y$ e os caches

In [53]:
# GRADED FUNCTION: rnn_forward

def rnn_forward(x, a0, parameters):
    """
    Implementar a propagação para frente da rede neural recorrente descrita na Figura (3).

    Argumentos:
    x -- dado de entrada para cada passo de tempo , de dimensões (n_x, m, T_x).
    a0 -- estado incial oculto, de dimensões (n_a, m)
    parameters -- dicionário python contendo:
                    Wax -- Matriz de Pesos que multipliaca a entrada (input), vetor numpy de dimensões (n_a, n_x)
                    Waa -- Matriz de Pesos que multipliaca o estado escondido, vetor numpy de dimensões (n_a, n_a)
                    Wya -- Matriz de Pesos que relaciona o estado escondido com a saída (output), vetor numpy de dimensões (n_y, n_a)
                    ba --  Bias, vetor numpy de dimensões (n_a, 1)
                    by -- Bias que relacion o estado escondido com a saída (output), vetor numpy de dimensões (n_y, 1)

    Retornos:
    a -- Estados escondidos para cada passo de tempo, array numpy de dimensões (n_a, m, T_x)
    y_pred -- Previsões para cada passo de tempo, array numpy de dimensões (n_y, m, T_x)
    caches -- Tuplas de valores necessários para o passo pra trás, contendo (list of caches, x)
    """
    
    # Inicializa "caches" que conterá a lista de todos os caches
    caches = []
    
    # Recupe as dimensões do formato de x e  parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### INICIE SEU CÓDIGO AQUI ###
    
    # inicialize "a" e "y" com zeros (≈2 lines)
    a = np.zeros((n_a, m, T_x), dtype=float)
    y_pred = np.zeros((n_y, m, T_x), dtype=float)
    
    # Inicialize a_next (≈1 linha)
    a_next = a0
    # Itere sobre todos os passos de tempo
    for t in range(T_x):
        # Atualize o próximo estado oculto, compute a previsão, pegue o cache (≈1 linha)
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        # Salve o valor do novo estado oculto "next" em a (≈1 linha)
        a[:,:,t] = a_next
        # Salve o valor da previsão em y (≈1 linha)
        y_pred[:,:,t] = yt_pred
        # Concatene (append) "cache" em "caches" (≈1 linha)
        caches.append(cache)
        
    ### FIM DO SEU CÓDIGO ###
    
    # Aramzene valores necessários para propagação para trás no cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [54]:
np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1] = ", a[4][1])
print("a.shape = ", a.shape)
print("y_pred[1][3] =", y_pred[1][3])
print("y_pred.shape = ", y_pred.shape)
print("caches[1][1][3] =", caches[1][1][3])
print("len(caches) = ", len(caches))

a[4][1] =  [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape =  (5, 10, 4)
y_pred[1][3] = [ 0.79560373  0.86224861  0.11118257  0.81515947]
y_pred.shape =  (2, 10, 4)
caches[1][1][3] = [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) =  2


**Saída Esperada**:

<table>
    <tr>
        <td>
            **a[4][1]**:
        </td>
        <td>
           [-0.99999375  0.77911235 -0.99861469 -0.99833267]
        </td>
    </tr>
        <tr>
        <td>
            **a.shape**:
        </td>
        <td>
           (5, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **y[1][3]**:
        </td>
        <td>
           [ 0.79560373  0.86224861  0.11118257  0.81515947]
        </td>
    </tr>
        <tr>
        <td>
            **y.shape**:
        </td>
        <td>
           (2, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **cache[1][1][3]**:
        </td>
        <td>
           [-1.1425182  -0.34934272 -0.20889423  0.58662319]
        </td>
    </tr>
        <tr>
        <td>
            **len(cache)**:
        </td>
        <td>
           2
        </td>
    </tr>

</table>

Parabéns! Você consegui construir um propagação para frente de uma rede neural recorrente do zero. Este trabalho servirá para várias aplicações, Mas sofre do famoso problema do gradiente que desaparece (vanishing gradient problems). Então, ele funciona melhor quando cada saída $y^{\langle t \rangle}$ pode ser estimada usando principalmente contexto "local"  (significa que a informações das entradas $x^{\langle t' \rangle}$ onde  $t'$ não é tão distante de $t$). 

Na próxima estapa, você irá desenvolver um modelo mais complexo chamado LSTM, que endereça melhor o problema do gradiente que desaparece. O modelo LSTM será mais capaz de lembrar a peça de informação mantida salva por vários passos de tempo. 